In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Border, Side

file_path = "/content/table_44.xlsx"
df = pd.read_excel(file_path)

In [ ]:
def drop_specific_rows(df):
    # Convert all columns to string to avoid issues with non-string values
    df = df.applymap(str)

    # Filter out rows that contain "Department" or "Total" in any column
    # mask = df.apply(lambda row: row.str.startswith(('Department', 'Total')).any(), axis=1)
    mask = df.apply(lambda row: row.str.startswith(('Total')).any(), axis=1)
    df = df[~mask]

    return df

# Apply the function
df = drop_specific_rows(df)

In [ ]:
def move_rows_to_end(df):
    # Identify rows that start with "Department" or "Emp Code"
    condition = df.iloc[:, 1].astype(str).str.startswith(("Department", "Emp Code"))

    # Extract these rows
    rows_to_move = df[condition]

    # Extract the remaining rows
    remaining_rows = df[~condition]

    # Concatenate the remaining rows with the rows to move at the end
    result_df = pd.concat([remaining_rows, rows_to_move], ignore_index=True)

    # Update the original DataFrame in place
    df.iloc[:] = result_df

# Apply the function
move_rows_to_end(df)

# Save the updated DataFrame to the original Excel file
file_path = "/content/ad_tb.xlsx"
df.to_excel(file_path, index=False)

In [ ]:
df = pd.read_excel(file_path, header=1)

In [ ]:
df = df.drop([col for col in df.columns if col.startswith('Unnamed')],axis=1)

In [ ]:
columns_to_keep = ["Att. Date", "InTime", "OutTime", "Shift", "S. InTime", "S. OutTime", "Punch Records"]

# Keep only the specified columns
df = df[columns_to_keep]

In [ ]:
df = pd.DataFrame(df)

df['Records'] = df['Punch Records']

In [ ]:
def change_name(records):
    if pd.isna(records):
        return records

    # Replace 'BD' with 'ED'
    entries = records.split(',')
    entries = [entry.replace('BD', 'ED') for entry in entries]

    # Replace 'Main Entrance' with 'ED' and 'Exit' with 'ED'
    entries = [entry.replace('Main Entrance', 'ED').replace('Exit', 'ED') for entry in entries]

    return ', '.join(entries)

# Apply the function to 'Records' column
df['Records'] = df['Records'].apply(change_name)

In [ ]:
def update_in_out_times(row):
    records = row["Punch Records"]

    if pd.isna(records):
        return pd.Series({'InTime': ' ', 'OutTime': ' '})

    entries = records.split(',')

    # Extract the first entry's time
    in_time_matches = re.findall(r"\d{2}:\d{2}", entries[0]) if entries else []
    in_time = in_time_matches[0] if in_time_matches else ' '

    # Extract the last entry's time and check if it's 'out'
    last_entry = entries[-2] if len(entries) > 1 else ' '
    out_time_matches = re.findall(r"\d{2}:\d{2}", last_entry) if last_entry else []
    out_time = out_time_matches[0] if out_time_matches else ' '

    if 'out' not in last_entry:
        out_time += ", records missing"

    return pd.Series({'InTime': in_time, 'OutTime': out_time})

# Apply the function to the DataFrame
df_filtered = df[~df['Att. Date'].str.startswith(('Emp Code', 'Department'))]

# Apply the update_in_out_times function to the filtered DataFrame
df_filtered[['InTime', 'OutTime']] = df_filtered.apply(update_in_out_times, axis=1)

# Update the original DataFrame with the results
df.update(df_filtered[['InTime', 'OutTime']])

# Fill missing values in the original DataFrame
df['InTime'] = df['InTime'].fillna(' ')
df['OutTime'] = df['OutTime'].fillna(' ')

<ipython-input-327-fbc0f0a70d32>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['InTime', 'OutTime']] = df_filtered.apply(update_in_out_times, axis=1)


In [ ]:
def remove_1st_entries(record):
    if pd.isna(record):
        return record

    entries = record.split(', ')
    filtered_entries = [entry for entry in entries if "1st" not in entry]
    return ', '.join(filtered_entries)

# Apply the function to the "Punch Records" column
df['Records'] = df['Records'].apply(remove_1st_entries)


In [ ]:
def filter_punch_records(record):
    if pd.isna(record):
        return record

    entries = record.split(',')

    valid_entries = [entry for entry in entries if ('in' in entry or 'out' in entry)]

    return ','.join(valid_entries)

df['Records'] = df['Records'].apply(filter_punch_records)

In [ ]:
df['Corrected Records'] = df['Records']

In [ ]:
def calculate_duration(row):
    in_time_str = row['InTime']
    out_time_str = row['OutTime'].replace(", records missing", "").strip()

    if in_time_str == ' ' or out_time_str == ' ':
        return ' '

    try:
        in_time = datetime.strptime(in_time_str, '%H:%M')
        out_time = datetime.strptime(out_time_str, '%H:%M')

        # Handle cases where out_time is past midnight (next day)
        if out_time < in_time:
            out_time += pd.DateOffset(days=1)

        duration = out_time - in_time
        total_minutes = duration.total_seconds() / 60
        hours, minutes = divmod(total_minutes, 60)
        return f"{int(hours)}h {int(minutes)}m"
    except Exception as e:
        return ' '

df['Total Duration'] = df.apply(calculate_duration, axis=1)

In [ ]:
df['Punch Records'].replace('NaN', pd.NA, inplace=True)
df['Records'].replace('NaN', pd.NA, inplace=True)

# Adding the "Status" column based on the "Records" column
df['Employee Status'] = df['Records'].apply(lambda x: 'Present' if pd.notna(x) and x != '' else 'Absent')

In [ ]:
def update_status_based_on_records(records, punch_records):
    if pd.isna(records) or records.strip() == '':
        return 'Absent'

    entries = records.split(', ')

    if len(entries) == 1:
        return 'Punch records missing'

    if 'out' in entries[0]:
        return 'Punch records missing'

    if len(entries) % 2 != 0:
        return 'Punch records missing'

    # Check for consecutive 'in' or 'out' entries
    for i in range(1, len(entries)):
        if ('in' in entries[i] and 'in' in entries[i-1]) or \
           ('out' in entries[i] and 'out' in entries[i-1]):
            return 'Punch records missing'

    return 'Valid Records'

# Function to mark columns empty based on specific words
def mark_columns_empty(row):
    words_to_check = ['Att. Date', 'InTime', 'OutTime', 'Shift', 'S. InTime', 'S. OutTime',
                      'Punch Records', 'Records']

    if row.name > 0:  # Check if row index is greater than 0 (excluding headers)
        for word in words_to_check:
            if pd.notna(row[word]) and any(word in str(cell) for cell in row):
                row['Employee Status'] = " "
                row['Records Status'] = " "
                row['Break Time'] = " "
                return row
    return row

# Ensure the columns 'Employee Status', 'Records Status', and 'Break Time' exist in the DataFrame
for col in ['Employee Status', 'Records Status', 'Break Time']:
    if col not in df.columns:
        df[col] = ""

# Apply the function to update the columns
df = df.apply(mark_columns_empty, axis=1)

# Apply the function to update 'Records Status' column
df['Records Status'] = df.apply(lambda row: update_status_based_on_records(row['Records'], row['Punch Records']), axis=1)

# Columns to check for NaN or words and alphabets
cols_to_check = ['Att. Date', 'InTime', 'OutTime', 'Shift', 'S. InTime', 'S. OutTime',
                 'Punch Records', 'Records']

def check_nan_and_update_status(row, cols_to_check):
    if all(pd.isna(row[col]) for col in cols_to_check):
        row['Employee Status'] = " "
        row['Records Status'] = " "
        row['Break Time'] = " "
    return row

# Apply the function to update 'Employee Status' column
df = df.apply(lambda row: check_nan_and_update_status(row, cols_to_check), axis=1)

# Reorder columns to ensure "Employee Status", "Records Status", and "Break Time (minutes)" are last
columns_order = [col for col in df.columns if col not in ['Employee Status', 'Records Status', 'Break Time']]
columns_order += ['Employee Status', 'Records Status', 'Break Time']
df = df[columns_order]

In [ ]:
df['Records_Dup'] = df['Records']

# Step 2: Define the function to check and adjust entries
def check_and_adjust_entries(records_dup):
    if pd.isna(records_dup):
        return '', 'N/A'

    entries = str(records_dup).split(', ')
    if len(entries) % 2 != 0:
        if entries[0].endswith("in(ED)") and entries[-1].endswith("in(ED)"):
            entries.append('--:--:out(ED)')
            return ', '.join(entries), 'Partially valid'

    for i in range(1, len(entries)):
        if (entries[i].endswith("in(ED)") and entries[i-1].endswith("in(ED)")) or (entries[i].endswith("out(ED)") and entries[i-1].endswith("out(ED)")):
            return ', '.join(entries), 'Invalid Records'

    return ', '.join(entries), 'Present'

# Step 3: Apply the function to the 'Records_Dup' column
df['Records_Dup'], df['Validity'] = zip(*df['Records_Dup'].apply(check_and_adjust_entries))

df['Corrected Records'] = df['Records']

# Step 4: Update the "Approx. Break Time" column based on the results
def update_approx_break_time(row):
    if row['Employee Status'] == 'Absent':
        return 'N/A'
    elif row['Records Status'] == 'Valid Records':
        return ''
    elif row['Validity'] == 'Partially valid':
        return 'Partially valid'
    elif row['Validity'] == 'Invalid Records':
        return 'Invalid Records'
    else:
        return 'N/A'

df['Approx. Break Time'] = df.apply(update_approx_break_time, axis=1)

# Function to remove the first "in" record and the last "out" record
def remove_first_in_last_out(records):
    entries = records.split(', ')
    if len(entries) > 0 and entries[0].endswith('in(ED)'):
        entries.pop(0)
    if len(entries) > 0 and entries[-1].endswith('out(ED)'):
        entries.pop(-1)
    return ', '.join(entries)

# Apply the function to the 'Records_Dup' column
df['Records_Dup'] = df['Records_Dup'].apply(lambda x: remove_first_in_last_out(x) if pd.notna(x) else x)

# Function to calculate break time for multiple records
def calculate_break_time(record):
    entries = record.split(', ')
    total_break_time = 0

    if len(entries) % 2 != 0:
        return 'Invalid entry length'

    for i in range(1, len(entries), 2):
        out_time_str = entries[i - 1].split(':out(ED)')[0].strip()
        in_time_str = entries[i].split(':in(ED)')[0].strip()

        out_time_match = re.search(r'\d{2}:\d{2}', out_time_str)
        in_time_match = re.search(r'\d{2}:\d{2}', in_time_str)

        if out_time_match and in_time_match:
            out_time = pd.to_datetime(out_time_match.group(), format='%H:%M')
            in_time = pd.to_datetime(in_time_match.group(), format='%H:%M')
            if in_time < out_time:
                in_time += pd.Timedelta(days=1)
            break_duration = in_time - out_time
            total_break_time += break_duration.total_seconds() / 60
        else:
            return 'Invalid time format'

    return int(total_break_time)

# Function to format break time
def format_break_time(minutes):
    if isinstance(minutes, str):
        return minutes
    hours = minutes // 60
    mins = minutes % 60
    if hours > 0:
        return f"{hours} hr {mins} mins" if mins > 0 else f"{hours} hr"
    else:
        return f"{mins} mins"

# Function to update Approx. Break Time column
def final_update_approx_break_time(row):
    if 'Partially valid' in row['Approx. Break Time']:
        break_time_minutes = calculate_break_time(row['Records_Dup'])
        formatted_break_time = format_break_time(break_time_minutes)

        if row['Employee Status'] == 'Absent':
            return 'N/A'
        elif row['Records Status'] == 'Valid Records':
            return formatted_break_time
        elif 'Partially valid' in row['Approx. Break Time']:
            return f"Partially valid, {formatted_break_time}"
        elif 'Invalid Records' in row['Approx. Break Time']:
            return 'Invalid Records'
        else:
            return formatted_break_time
    else:
        return row['Approx. Break Time']

# Apply the final update function to the DataFrame
df['Approx. Break Time'] = df.apply(final_update_approx_break_time, axis=1)

In [ ]:
def handle_invalid_entries(approx_break_time):
    if pd.isna(approx_break_time):
        return approx_break_time
    if "Partially valid, Invalid entry length" in approx_break_time:
        return "Invalid Entries, Punch missed"
    if re.search(r'-\d+', approx_break_time):
        return "Invalid Entries, Punch missed"
    return approx_break_time

# Apply the function to the "Approx. Break Time" column
df['Approx. Break Time'] = df['Approx. Break Time'].apply(handle_invalid_entries)

In [ ]:
def remove_first_in_last_out(records):
    entries = records.split(', ')
    if len(entries) > 0 and entries[0].endswith('in(ED)'):
        entries.pop(0)
    if len(entries) > 0 and entries[-1].endswith('out(ED)'):
        entries.pop(-1)
    return ', '.join(entries)

# Apply the function to the 'Records' column
df['Records'] = df['Records'].apply(lambda x: remove_first_in_last_out(x) if pd.notna(x) else x)


In [ ]:
def calculate_break_time(row):
    if row['Employee Status'] == 'Absent':
        return 'N/A'

    if row['Employee Status'] == 'Present':
        entries = row['Records'].split(',')
        total_break_time = 0
        for i in range(1, len(entries), 2):
            in_time_str = entries[i - 1].split()[-1]
            out_time_str = entries[i].split()[-1]

            in_time_match = re.search(r'\d{2}:\d{2}', in_time_str)
            out_time_match = re.search(r'\d{2}:\d{2}', out_time_str)

            if in_time_match and out_time_match:
                in_time = pd.to_datetime(in_time_match.group(), format='%H:%M')
                out_time = pd.to_datetime(out_time_match.group(), format='%H:%M')
                break_duration = out_time - in_time
                total_break_time += break_duration.total_seconds() / 60

            if row['Records Status'] == 'Punch records missing':
                return 'N/A'

        return int(total_break_time)
    return 0

# Function to format break time
def format_break_time(minutes):
    if minutes == 'N/A':
        return minutes
    hours = minutes // 60
    mins = minutes % 60
    if hours > 0:
        return f"{hours} hr {mins} mins" if mins > 0 else f"{hours} hr"
    else:
        return f"{mins} mins"

# Apply the functions to the DataFrame
df['Break Time'] = df.apply(calculate_break_time, axis=1)
df['Break Time'] = df['Break Time'].apply(format_break_time)

In [ ]:
def update_break_time_for_missing_records(row):
    if row['Records Status'] == 'Punch records missing':
        return 'N/A'
    return row['Break Time']

# Apply the function to the DataFrame
df['Break Time'] = df.apply(update_break_time_for_missing_records, axis=1)

In [ ]:
df['Punch Records'].replace('NaN', pd.NA, inplace=True)
df['Records'].replace('NaN', pd.NA, inplace=True)

# Adding the "Status" column based on the "Records" column
df['Employee Status'] = df['Records'].apply(lambda x: 'Present' if pd.notna(x) and x != '' else 'Absent')


In [ ]:
# List of columns to drop
columns_to_drop = ['Records_Dup','Validity','Records']

# Strip leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

# Drop the specified columns
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [ ]:
def mark_columns_empty(row):
    words_to_check = ['Att. Date', 'InTime', 'OutTime', 'Shift', 'S. InTime', 'S. OutTime',
                      'Punch Records', 'Corrected Records']

    if row.name > 0:  # Check if row index is greater than 0 (excluding headers)
        for word in words_to_check:
            if word in row and pd.notna(row[word]) and any(word in str(cell) for cell in row):
                row['Employee Status'] = " "
                row['Records Status'] = " "
                row['Break Time'] = " "
                return row
    return row

# Function to update 'Records Status' column
def update_status_based_on_records(records, punch_records):
    if pd.isna(records) or records.strip() == '':
        return 'Absent'

    entries = records.split(', ')

    if len(entries) == 1:
        return 'Punch records missing'

    if 'out' in entries[0]:
        return 'Punch records missing'

    if len(entries) % 2 != 0:
        return 'Punch records missing'

    # Check for consecutive 'in' or 'out' entries
    for i in range(1, len(entries)):
        if ('in' in entries[i] and 'in' in entries[i-1]) or \
           ('out' in entries[i] and 'out' in entries[i-1]):
            return 'Punch records missing'

    return 'Valid Records'

# Apply the function to update 'Records Status' column
if 'Records' in df.columns and 'Punch Records' in df.columns:
    df['Records Status'] = df.apply(lambda row: update_status_based_on_records(row['Records'], row['Punch Records']), axis=1)

# Columns to check for NaN or words and alphabets
cols_to_check = ['Att. Date', 'InTime', 'OutTime', 'Shift', 'S. InTime', 'S. OutTime',
                 'Punch Records', 'Records']

def check_nan_and_update_status(row, cols_to_check):
    # If the row starts with 'department' or 'emp code', set 'Employee Status', 'Records Status' and 'Break Time' to empty space
    if any(str(row[col]).strip().lower().startswith(('department', 'emp code')) for col in row.index if pd.notna(row[col])):
        row['Employee Status'] = " "
        row['Records Status'] = " "
        row['Break Time'] = " "
        row['Approx. Break Time'] = " "
        row['Total Duration'] = " "
    # If the entire row is NaN, set 'Employee Status', 'Records Status' and 'Break Time' to empty space
    # # elif row.isna().all():
    # #     row['Employee Status'] = " "
    #     row['Records Status'] = " "
    #     row['Break Time'] = " "
    # # If 'Records' column is empty, set 'Employee Status' and 'Break Time' to empty space
    # elif 'Records' in row and (pd.isna(row['Records']) or row['Records'].strip() == ''):
    #     row['Employee Status'] = " "
    #     row['Break Time'] = " "
    # # If 'Records Status' is empty, set 'Employee Status' and 'Break Time' to empty space
    # elif 'Records Status' in row and row['Records Status'].strip() == '':
    #     row['Employee Status'] = " "
    #     row['Break Time'] = " "
    else:
        if 'Punch Records' in row and pd.notna(row['Punch Records']) and row['Punch Records'].strip() != '':
            row['Employee Status'] = "Present"
        else:
            row['Employee Status'] = "Absent"
    return row

# Apply the function to update 'Employee Status', 'Records Status', and 'Break Time' columns
df = df.apply(lambda row: check_nan_and_update_status(row, cols_to_check), axis=1)

# Apply the function to mark columns empty based on specific words
df = df.apply(mark_columns_empty, axis=1)

# Reorder columns to ensure "Employee Status", "Records Status", and "Break Time" are last
columns_order = [col for col in df.columns if col not in ['Corrected Records', 'Records Status','Total Duration', 'Employee Status', 'Break Time', 'Approx. Break Time']]
columns_order += ['Corrected Records', 'Records Status', 'Total Duration', 'Employee Status', 'Break Time', 'Approx. Break Time']
df = df[columns_order]


In [ ]:
def should_drop_row(row):
    first_cell_value = str(row.iloc[0])
    return first_cell_value.startswith(('Total'))

# Apply the function to filter out rows
df = df[~df.apply(should_drop_row, axis=1)]

In [ ]:
leave_dates_row = {
    'Att. Date': 'Leave Dates:',
    'InTime': leave_dates[0],  # Dates in the second column
    'OutTime': 'No. of leaves:',
    'Shift': total_leaves,
    'S. InTime': '',
    'S. OutTime': '',
    'Punch Records': '',
    'Corrected Records': '',
    'Records Status': '',
    'Total Duration': '',
    'Employee Status': '',
    'Break Time': '',
    'Approx. Break Time': ''  # Count in the last column
}

# Convert leave_dates_row to a DataFrame
leave_dates_df = pd.DataFrame([leave_dates_row])

# Find the index to insert the new rows
emp_code_index = df.index[df['Att. Date'].str.contains('Emp Code:', na=False)].tolist()[0]

# Insert the new row after "Emp Code"
df = pd.concat([df.iloc[:emp_code_index + 1], leave_dates_df, df.iloc[emp_code_index + 1:]], ignore_index=True)


In [ ]:
temp_file_path = 'temp_file.xlsx'
df.to_excel(temp_file_path, index=False)

# Load workbook and select active sheet
wb = load_workbook(temp_file_path)
ws = wb.active

# Define fill colors
aqua_fill = PatternFill(start_color="CCDCF8", end_color="CCDCF8", fill_type="solid")
f8c9eb_fill = PatternFill(start_color="F9D3EE", end_color="F9D3EE", fill_type="solid")
head_fill = PatternFill(start_color="DBFBEA", end_color="DBFBEA", fill_type="solid")
specific_fill = PatternFill(start_color="D9C5E9", end_color="D9C5E9", fill_type="solid")

# Apply the fill color to cells with the specified keywords

# Define border style
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

# Get the index of the "Break Time (minutes)" and "Approx. Break Time" columns
break_time_col_idx = df.columns.get_loc('Break Time') + 1
approx_break_time_col_idx = df.columns.get_loc('Approx. Break Time') + 1

# Apply aqua color to the "Break Time (minutes)" column
for row in ws.iter_rows(min_row=2, min_col=break_time_col_idx, max_col=break_time_col_idx):
    for cell in row:
        cell.fill = aqua_fill
        cell.border = thin_border


# Apply f8c9eb color to the "Approx. Break Time" column
for row in ws.iter_rows(min_row=2, min_col=approx_break_time_col_idx, max_col=approx_break_time_col_idx):
    for cell in row:
        cell.fill = f8c9eb_fill
        cell.border = thin_border


for cell in ws[1]:
    cell.fill = head_fill


    # Function to fill specific cells with the defined color
def fill_specific_cells(ws, keywords, fill):
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            if cell.value in keywords:
                cell.fill = fill
                cell.border = thin_border

# Keywords to look for
keywords = ["Employee Name :", "Department:", "Emp Code:", "Leave Dates:", "No. of leaves:"]
fill_specific_cells(ws, keywords, specific_fill)


# Set the height of each row to 100px (approximately 75 points)
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    ws.row_dimensions[row[0].row].height = 33.60  # 100px is approximately 75 points

# Define columns and their respective widths
column_widths = {
    'InTime': 14.29,  # 100px
    'OutTime': 22.56,  # 100px
    'Shift': 14.29,  # 100px
    'S. InTime': 14.29,  # 100px
    'S. OutTime': 14.29,  # 100px
    'Punch Records': 38.57,  # 270px
    'Corrected Records': 38.57,  # 270px
    'Approx. Break Time': 38.57,  # 270px
    'Break Time': 16.00,  # 270px
    'Total Duration': 16.00,  # 270px
    'Att. Date': 21.44,  # 200px
    'Employee Status': 21.44,  # 200px
    'Records Status': 21.44  # 200px
}

# Set the column widths
for col_name, width in column_widths.items():
    if col_name in df.columns:
        col_idx = df.columns.get_loc(col_name) + 1
        col_letter = ws.cell(row=1, column=col_idx).column_letter
        ws.column_dimensions[col_letter].width = width

# Center-align all cells horizontally and vertically
alignment = Alignment(horizontal='center', vertical='center')

for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.alignment = alignment

# Locate the employee name in the worksheet
employee_name = None
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        if cell.value == "Employee Name :":
            employee_name = cell.offset(column=3).value  # Get the value of the next cell in the same row
            break
    if employee_name:
        break

# Use a default name if employee name is not found
if not employee_name:
    employee_name = "Unnamed_Employee"

output_file_name = f"{employee_name}.xlsx"

wb.save(output_file_name)

In [ ]:
df

,Att. Date,InTime,OutTime,Shift,S. InTime,S. OutTime,Punch Records,Corrected Records,Records Status,Total Duration,Employee Status,Break Time,Approx. Break Time
0,21-Jun-2024,10:23,"14:32, records missing",Sudhev,12:00,17:00,"10:23:in(Main Entrance),10:23:(1st),11:30:out(...","10:23:in(ED), 11:30:out(ED), 12:03:in(ED), 13:...",Punch records missing,4h 9m,Present,N/A,"Partially valid, 1 hr 14 mins"
1,22-Jun-2024,,,Sudhev,12:00,17:00,NaN,NaN,Absent,,Absent,N/A,N/A
2,23-Jun-2024,,,Sudhev,12:00,17:00,NaN,NaN,Absent,,Absent,N/A,N/A
3,24-Jun-2024,09:23,14:51,Sudhev,12:00,17:00,"09:23:in(Main Entrance),09:40:out(Main Entranc...","09:23:in(ED), 09:40:out(ED), 10:07:in(ED), 14:...",Valid Records,5h 28m,Present,27 mins,
4,25-Jun-2024,10:51,"20:56, records missing",Sudhev,12:00,17:00,"10:51:in(Main Entrance),12:54:out(Main Entranc...","10:51:in(ED), 12:54:out(ED), 13:19:in(ED), 14:...",Punch records missing,10h 5m,Present,N/A,"Partially valid, 4 hr 4 mins"
5,26-Jun-2024,,,Sudhev,12:00,17:00,NaN,NaN,Absent,,Absent,N/A,N/A
6,27-Jun-2024,,,Sudhev,12:00,17:00,NaN,NaN,Absent,,Absent,N/A,N/A
7,28-Jun-2024,10:35,"10:38, records missing",Sudhev,12:00,17:00,"10:35:in(Main Entrance),10:38:(Main Entrance),",10:35:in(ED),Punch records missing,0h 3m,Present,N/A,"Invalid Entries, Punch missed"
8,Emp Code:,052,Employee Name :,NaN,NaN,Muthu,NaN,NaN,,,,,
9,Leave Dates:,28-Jun-2024,No. of leaves:,1,,,,,,,,,


In [ ]:
from google.colab import files
files.download(output_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>